## 1. MH data load

In [1]:
import torch
import torch.nn as nn
import torchvision

import numpy as np
from PIL import Image

import cv2
import os
import numpy as np

from dataset_MH import *
from googlenet import *

print("torch-version:", torch.__version__)
print("Available GPU:", torch.cuda.current_device())

torch-version: 1.1.0
Available GPU: 0


In [2]:
# Params
input_size = (30, 30) #1200x800
batch_size = 10 #12
num_workers = 10 #12

# 기존 데이터 경로
NORMAL_DIR = '../snapshots_pureok'
FAULT_DIR = '../snapshots'

# 추가 데이터 경로
ADD_NORMAL_DIR = '../snapshots_punch181'
ADD_FAULT_DIR = '../snapshots_punch181_NG'

# 카메라 번호 (1~4까지 있음, 카메라별로 각각 다 따로 학습해야됩니다.) CAM1 / CAM2 / CAM3 / CAM4
cam1 = 'CAM1'
cam2 = 'CAM2'
cam3 = 'CAM3'
cam4 = 'CAM4'

In [3]:
# 기존 데이터 로드
normal_list1 = filename_list(NORMAL_DIR, cam=cam1)
fault_list1 = filename_list(FAULT_DIR, cam=cam1)
normal_list2 = filename_list(NORMAL_DIR, cam=cam2)
fault_list2 = filename_list(FAULT_DIR, cam=cam2)
normal_list3 = filename_list(NORMAL_DIR, cam=cam3)
fault_list3 = filename_list(FAULT_DIR, cam=cam3)
normal_list4 = filename_list(NORMAL_DIR, cam=cam4)
fault_list4 = filename_list(FAULT_DIR, cam=cam4)

mhdb1 = MyungHwa_ansan_db(normal_list1, fault_list1, dataType='train', input_size=input_size, augmentation=True)
mhdb2 = MyungHwa_ansan_db(normal_list2, fault_list2, dataType='train', input_size=input_size, augmentation=True)
mhdb3 = MyungHwa_ansan_db(normal_list3, fault_list3, dataType='train', input_size=input_size, augmentation=True)
mhdb4 = MyungHwa_ansan_db(normal_list4, fault_list4, dataType='train', input_size=input_size, augmentation=True)

mhdb5 = MyungHwa_ansan_db(normal_list1, fault_list1, dataType='val', input_size=input_size, augmentation=True)
mhdb6 = MyungHwa_ansan_db(normal_list2, fault_list2, dataType='val', input_size=input_size, augmentation=True)
mhdb7 = MyungHwa_ansan_db(normal_list3, fault_list3, dataType='val', input_size=input_size, augmentation=True)
mhdb8 = MyungHwa_ansan_db(normal_list4, fault_list4, dataType='val', input_size=input_size, augmentation=True)
   
# 추가 데이터 로드
add_normal_list1 = filename_list(ADD_NORMAL_DIR, cam=cam1)
add_fault_list1 = filename_list(ADD_FAULT_DIR, cam=cam1)
add_normal_list2 = filename_list(ADD_NORMAL_DIR, cam=cam2)
add_fault_list2 = filename_list(ADD_FAULT_DIR, cam=cam2)
add_normal_list3 = filename_list(ADD_NORMAL_DIR, cam=cam3)
add_fault_list3 = filename_list(ADD_FAULT_DIR, cam=cam3)
add_normal_list4 = filename_list(ADD_NORMAL_DIR, cam=cam4)
add_fault_list4 = filename_list(ADD_FAULT_DIR, cam=cam4)

add_mhdb1 = MyungHwa_ansan_db(add_normal_list1, add_fault_list1, dataType='train', input_size=input_size, augmentation=True)
add_mhdb2 = MyungHwa_ansan_db(add_normal_list2, add_fault_list2, dataType='train', input_size=input_size, augmentation=True)
add_mhdb3 = MyungHwa_ansan_db(add_normal_list3, add_fault_list3, dataType='train', input_size=input_size, augmentation=True)
add_mhdb4 = MyungHwa_ansan_db(add_normal_list4, add_fault_list4, dataType='train', input_size=input_size, augmentation=True)

add_mhdb5 = MyungHwa_ansan_db(add_normal_list1, add_fault_list1, dataType='val', input_size=input_size, augmentation=True)
add_mhdb6 = MyungHwa_ansan_db(add_normal_list2, add_fault_list2, dataType='val', input_size=input_size, augmentation=True)
add_mhdb7 = MyungHwa_ansan_db(add_normal_list3, add_fault_list3, dataType='val', input_size=input_size, augmentation=True)
add_mhdb8 = MyungHwa_ansan_db(add_normal_list4, add_fault_list4, dataType='val', input_size=input_size, augmentation=True)
    
# 데이터셋 합치기
concat_mhdb1 = torch.utils.data.ConcatDataset([mhdb1, add_mhdb1])
concat_mhdb2 = torch.utils.data.ConcatDataset([mhdb2, add_mhdb2])
concat_mhdb3 = torch.utils.data.ConcatDataset([mhdb3, add_mhdb3])
concat_mhdb4 = torch.utils.data.ConcatDataset([mhdb4, add_mhdb4])

concat_mhdb5 = torch.utils.data.ConcatDataset([mhdb5, add_mhdb5])
concat_mhdb6 = torch.utils.data.ConcatDataset([mhdb6, add_mhdb6])
concat_mhdb7 = torch.utils.data.ConcatDataset([mhdb7, add_mhdb7])
concat_mhdb8 = torch.utils.data.ConcatDataset([mhdb8, add_mhdb8])

In [4]:
mhdb_batch1 = torch.utils.data.DataLoader(concat_mhdb1, batch_size=batch_size, shuffle=True, num_workers=num_workers)
mhdb_batch2 = torch.utils.data.DataLoader(concat_mhdb2, batch_size=batch_size, shuffle=True, num_workers=num_workers)
mhdb_batch3 = torch.utils.data.DataLoader(concat_mhdb3, batch_size=batch_size, shuffle=True, num_workers=num_workers)
mhdb_batch4 = torch.utils.data.DataLoader(concat_mhdb4, batch_size=batch_size, shuffle=True, num_workers=num_workers)

val_mhdb_batch1 = torch.utils.data.DataLoader(concat_mhdb5, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_mhdb_batch2 = torch.utils.data.DataLoader(concat_mhdb6, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_mhdb_batch3 = torch.utils.data.DataLoader(concat_mhdb7, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_mhdb_batch4 = torch.utils.data.DataLoader(concat_mhdb8, batch_size=batch_size, shuffle=True, num_workers=num_workers)
#print("mhdb batch size :", len(mhdb_batch2))

## 2. Model

In [5]:
#def __init__(self, in_channel=1, num_classes=1, aux_block=True):
model = GoogLeNet(in_channel=3).cuda()

for i in model.named_children():
    print(i)

('conv_block1', Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
))
('maxpool1', MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True))
('conv_block2', Sequential(
  (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (1): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
))
('conv_block3', Sequential(
  (0): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (1): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
))
('maxpool2', MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True))
('inception3a', InceptionBlock(
  (conv_1x1): Sequential(
    (0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): LeakyReLU(negative_slope=0.01, inplace)


## 3. Train

In [6]:
from torch.optim.lr_scheduler import StepLR

# train params
learning_rate = 1e-03 #5e-04
epoch = 3

# optimizer & loss func
loss_func = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
scheduler = StepLR(optimizer, step_size=8, gamma=0.04)

## dataset: mhdb_batch1 / mhdb_batch2 / mhdb_batch3 / mhdb_batch4 / val_mhdb_batch1 ...
for img, label in val_mhdb_batch1:
    print(img, label)

tensor([[[[0.1843, 0.1608, 0.1373,  ..., 0.6078, 0.6588, 0.7176],
          [0.1804, 0.1569, 0.1333,  ..., 0.3216, 0.3333, 0.3765],
          [0.1882, 0.1686, 0.1490,  ..., 0.3373, 0.3098, 0.2941],
          ...,
          [0.2196, 0.2157, 0.1961,  ..., 0.6824, 0.7412, 0.6588],
          [0.2078, 0.2039, 0.1373,  ..., 0.6314, 0.7373, 0.7333],
          [0.1843, 0.1922, 0.1255,  ..., 0.6471, 0.6980, 0.7412]],

         [[0.1843, 0.1608, 0.1373,  ..., 0.6078, 0.6588, 0.7176],
          [0.1804, 0.1569, 0.1333,  ..., 0.3216, 0.3333, 0.3765],
          [0.1882, 0.1686, 0.1490,  ..., 0.3333, 0.3098, 0.2941],
          ...,
          [0.2196, 0.2157, 0.1961,  ..., 0.6824, 0.7412, 0.6588],
          [0.2078, 0.2039, 0.1373,  ..., 0.6314, 0.7373, 0.7333],
          [0.1843, 0.1922, 0.1255,  ..., 0.6471, 0.6980, 0.7412]],

         [[0.1843, 0.1608, 0.1373,  ..., 0.6078, 0.6588, 0.7176],
          [0.1804, 0.1569, 0.1333,  ..., 0.3216, 0.3333, 0.3765],
          [0.1882, 0.1686, 0.1490,  ..., 0

tensor([[[[0.0941, 0.0118, 0.0039,  ..., 0.0314, 0.0275, 0.1059],
          [0.0235, 0.0078, 0.0039,  ..., 0.2588, 0.1882, 0.1255],
          [0.0118, 0.0078, 0.0078,  ..., 0.3804, 0.4078, 0.4275],
          ...,
          [0.0078, 0.0039, 0.0039,  ..., 0.0039, 0.0118, 0.0863],
          [0.0118, 0.0039, 0.0039,  ..., 0.0039, 0.0078, 0.0588],
          [0.0471, 0.0078, 0.0078,  ..., 0.0039, 0.0078, 0.0549]],

         [[0.0941, 0.0118, 0.0039,  ..., 0.0314, 0.0275, 0.1059],
          [0.0235, 0.0078, 0.0039,  ..., 0.2588, 0.1882, 0.1255],
          [0.0118, 0.0078, 0.0078,  ..., 0.3804, 0.4078, 0.4275],
          ...,
          [0.0078, 0.0039, 0.0039,  ..., 0.0039, 0.0118, 0.0863],
          [0.0118, 0.0039, 0.0039,  ..., 0.0039, 0.0078, 0.0588],
          [0.0471, 0.0078, 0.0078,  ..., 0.0039, 0.0078, 0.0549]],

         [[0.0941, 0.0118, 0.0039,  ..., 0.0314, 0.0275, 0.1059],
          [0.0235, 0.0078, 0.0039,  ..., 0.2588, 0.1882, 0.1255],
          [0.0118, 0.0078, 0.0078,  ..., 0

tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0235, 0.0157, 0.0039],
          [0.0000, 0.0000, 0.0000,  ..., 0.1529, 0.1490, 0.1373],
          [0.0000, 0.0000, 0.0000,  ..., 0.1686, 0.1608, 0.1490],
          ...,
          [0.0039, 0.0078, 0.0078,  ..., 0.0078, 0.0118, 0.0706],
          [0.0039, 0.0078, 0.0039,  ..., 0.0078, 0.0078, 0.0353],
          [0.0039, 0.0039, 0.0039,  ..., 0.0078, 0.0118, 0.0235]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0235, 0.0157, 0.0039],
          [0.0000, 0.0000, 0.0000,  ..., 0.1529, 0.1490, 0.1373],
          [0.0000, 0.0000, 0.0000,  ..., 0.1686, 0.1608, 0.1490],
          ...,
          [0.0039, 0.0078, 0.0078,  ..., 0.0078, 0.0118, 0.0706],
          [0.0039, 0.0078, 0.0039,  ..., 0.0078, 0.0078, 0.0353],
          [0.0039, 0.0039, 0.0039,  ..., 0.0078, 0.0118, 0.0235]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0235, 0.0157, 0.0039],
          [0.0000, 0.0000, 0.0000,  ..., 0.1529, 0.1490, 0.1373],
          [0.0000, 0.0000, 0.0000,  ..., 0

tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0

tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0

tensor([[[[0.1843, 0.1686, 0.1529,  ..., 0.6275, 0.6784, 0.7294],
          [0.1804, 0.1647, 0.1490,  ..., 0.3294, 0.3529, 0.4039],
          [0.1922, 0.1882, 0.1804,  ..., 0.3137, 0.2941, 0.2824],
          ...,
          [0.2078, 0.1843, 0.2353,  ..., 0.7020, 0.7412, 0.6353],
          [0.2118, 0.1922, 0.2431,  ..., 0.6431, 0.7333, 0.7098],
          [0.2392, 0.2196, 0.2510,  ..., 0.6549, 0.6706, 0.7176]],

         [[0.1843, 0.1686, 0.1529,  ..., 0.6275, 0.6784, 0.7294],
          [0.1804, 0.1647, 0.1490,  ..., 0.3294, 0.3529, 0.4039],
          [0.1922, 0.1882, 0.1804,  ..., 0.3137, 0.2941, 0.2824],
          ...,
          [0.2078, 0.1843, 0.2353,  ..., 0.7020, 0.7412, 0.6353],
          [0.2118, 0.1922, 0.2431,  ..., 0.6431, 0.7333, 0.7098],
          [0.2392, 0.2196, 0.2510,  ..., 0.6549, 0.6706, 0.7176]],

         [[0.1843, 0.1686, 0.1529,  ..., 0.6275, 0.6784, 0.7294],
          [0.1804, 0.1647, 0.1490,  ..., 0.3294, 0.3529, 0.4039],
          [0.1922, 0.1882, 0.1804,  ..., 0

tensor([[[[0.1843, 0.1647, 0.1412,  ..., 0.6275, 0.6824, 0.7333],
          [0.1804, 0.1569, 0.1373,  ..., 0.3216, 0.3490, 0.4039],
          [0.1922, 0.1765, 0.1647,  ..., 0.2745, 0.2745, 0.2745],
          ...,
          [0.2157, 0.1922, 0.2235,  ..., 0.6902, 0.7412, 0.6510],
          [0.2235, 0.2078, 0.2078,  ..., 0.6431, 0.7490, 0.7333],
          [0.2275, 0.2275, 0.2000,  ..., 0.6902, 0.7137, 0.7490]],

         [[0.1843, 0.1647, 0.1412,  ..., 0.6275, 0.6824, 0.7333],
          [0.1804, 0.1569, 0.1373,  ..., 0.3216, 0.3490, 0.4039],
          [0.1922, 0.1765, 0.1647,  ..., 0.2745, 0.2745, 0.2745],
          ...,
          [0.2157, 0.1922, 0.2235,  ..., 0.6902, 0.7412, 0.6510],
          [0.2235, 0.2078, 0.2078,  ..., 0.6431, 0.7490, 0.7333],
          [0.2275, 0.2275, 0.2000,  ..., 0.6902, 0.7137, 0.7490]],

         [[0.1843, 0.1647, 0.1412,  ..., 0.6275, 0.6824, 0.7333],
          [0.1804, 0.1569, 0.1373,  ..., 0.3216, 0.3490, 0.4039],
          [0.1922, 0.1765, 0.1647,  ..., 0

tensor([[[[0.5725, 0.5608, 0.5412,  ..., 0.2039, 0.1725, 0.1451],
          [0.5725, 0.5608, 0.5451,  ..., 0.3608, 0.3451, 0.3216],
          [0.5608, 0.5451, 0.5294,  ..., 0.3725, 0.3725, 0.3686],
          ...,
          [0.5098, 0.4549, 0.4588,  ..., 0.1882, 0.1765, 0.2588],
          [0.5137, 0.4706, 0.5059,  ..., 0.2078, 0.1843, 0.2706],
          [0.5294, 0.4863, 0.5137,  ..., 0.2196, 0.2314, 0.3176]],

         [[0.5725, 0.5608, 0.5412,  ..., 0.2039, 0.1725, 0.1451],
          [0.5725, 0.5608, 0.5451,  ..., 0.3608, 0.3451, 0.3216],
          [0.5608, 0.5451, 0.5294,  ..., 0.3725, 0.3725, 0.3686],
          ...,
          [0.5098, 0.4549, 0.4588,  ..., 0.1882, 0.1765, 0.2588],
          [0.5137, 0.4706, 0.5059,  ..., 0.2078, 0.1843, 0.2706],
          [0.5294, 0.4863, 0.5137,  ..., 0.2196, 0.2314, 0.3176]],

         [[0.5725, 0.5608, 0.5412,  ..., 0.2039, 0.1725, 0.1451],
          [0.5725, 0.5608, 0.5451,  ..., 0.3608, 0.3451, 0.3216],
          [0.5608, 0.5451, 0.5294,  ..., 0

tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0

In [7]:
def test(dataset, model, criterion):
    model.eval()
    running_loss = 0.0
    
    for img, label in dataset:
        img = img.cuda()
        label = label.cuda()
        
        with torch.no_grad():
            output = model(img)
            #print(output)
            loss = criterion(output, label)
            
        running_loss += loss.item()
    return running_loss

for i in range(epoch):
    model.train(True)
    train_loss = 0.0
    
    for img, label in mhdb_batch1:
        img = img.cuda()
        label = label.cuda()
        
        optimizer.zero_grad()
        output = model(img)
        #print(output)
        #print(label)
        
        loss_1 = loss_func(output[0], label) * 0.3
        #print(loss_1)
        loss_2 = loss_func(output[1], label) * 0.3
        #print(loss_2)
        loss_3 = loss_func(output[2], label)
        #print(loss_3)
        
        total_loss = loss_1 + loss_2 + loss_3
        train_loss += total_loss.item()
        total_loss.backward()
        scheduler.step()
        #print(total_loss)

    if i % 1 == 0:
        print("Epoch:", i)
        print("Train Loss:", train_loss)
        val_loss = test(val_mhdb_batch1, model, loss_func)
        print("Val Loss:", val_loss)
        
        name = "GoogLeNet-MH"+"-Epoch-"+"%s"%epoch+"-Loss-"+"%s"%str(val_loss)[:7]+".pth"
        model_path = './saved_models/' + name
        torch.save(model.state_dict(), model_path)
        print("Saved model:", model_path, "\n")
        

Epoch: 0
Train Loss: 284.23009490966797
Val Loss: 30.00900822877884
Saved model: ./saved_models/GoogLeNet-MH-Epoch-3-Loss-30.0090.pth 

Epoch: 1
Train Loss: 284.18631076812744
Val Loss: 30.000409066677094
Saved model: ./saved_models/GoogLeNet-MH-Epoch-3-Loss-30.0004.pth 

Epoch: 2
Train Loss: 284.21325623989105
Val Loss: 30.009008646011353
Saved model: ./saved_models/GoogLeNet-MH-Epoch-3-Loss-30.0090.pth 



## 4. Test